## Autoencoder Classifiers
**Author:** [westny](https://github.com/westny) <br>
**Date created:** 2021-05-04 <br>
**Last modified:** 2021-05-04 <br>
**Description:** Implementation ANNs to classify maneuvers using encoding provided by autoencoder. <br>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import OneHotEncoder

### Base model

In [ ]:
class AEClassifier(nn.Module):
    """ IntentClassifier. Classifies intent (LCL, LCR, LK) """
    def __init__(self, n_inputs=256, n_outputs=3):
        super(AEClassifier, self).__init__()
        self.n_inputs = n_inputs

        self.classification = nn.Sequential(
            nn.Linear(n_inputs, 1024),
            nn.BatchNorm1d(1024),
            nn.PReLU(),
            
            nn.Dropout(0.5),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            
            nn.Dropout(0.5),
            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.PReLU(),
            
            nn.Dropout(0.15),
            nn.Linear(64, n_outputs),
        )
    
    def forward(self, x, y):
        out = self.classification(x)
        return F.softmax(out, dim=1)

### Using categorical features

In [ ]:
class AEClassifierRule(nn.Module):
    """ IntentClassifier. Classifies intent (LCL, LCR, LK) """
    def __init__(self, n_inputs=256, n_outputs=3):
        super(AEClassifierRule, self).__init__()
        self.n_inputs = n_inputs
        self.static_feature_encoder = StaticFeatureEncoder()

        self.classification = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(n_inputs+11, 1024),
            nn.BatchNorm1d(1024),
            nn.PReLU(),
            
            nn.Dropout(0.5),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            
            nn.Dropout(0.5),
            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.PReLU(),
            
            nn.Dropout(0.15),
            nn.Linear(64, n_outputs),
        )
    
    def forward(self, x, y):
        static_out, condition = self.static_feature_encoder(y)
        x = torch.tanh(x)
        x = torch.cat((x, static_out), dim=1)
        out = self.classification(x)
        return F.softmax(out, dim=1)

### Categorical Feature encoder

In [ ]:
class CategoricalFeatureEncoder(nn.Module):
    """ Encodes categorical features """
    def __init__(self, n_features=3+2*2):
        super(CategoricalFeatureEncoder, self).__init__()
        
        self.n_features = n_features
        enc = OneHotEncoder()
        
        #lane_types= ['travel', 'through', 'express', 'aux']
        lane_types = np.array([1, 2, 3, 4]).reshape(-1, 1)
        self.lane_enc = torch.from_numpy(enc.fit_transform(lane_types).toarray())

        v_class_arr = np.array([1, 2, 3]).reshape(-1, 1)
        self.v_enc = torch.from_numpy(enc.fit_transform(v_class_arr).toarray())
        
        neighbor_lanes = np.array([0, 1]).reshape(-1, 1)
        self.lane_neigh = torch.from_numpy(enc.fit_transform(neighbor_lanes).toarray())
        
        self.left_lane_exists = torch.ones(2, 3, device='cuda')
        self.left_lane_exists[0,0] = 0
        
        self.right_lane_exists = torch.ones(2, 3, device='cuda')
        self.right_lane_exists[0,-1] = 0
        
    def get_lane_enc(self, lane_n):
        return self.lane_enc[lane_n-1, :]
    
    def get_left_lane_enc(self, lane_n):
        return self.lane_neigh[lane_n, :], self.left_lane_exists[lane_n, :]
    
    def get_right_lane_enc(self, lane_n):
        return self.lane_neigh[lane_n, :], self.right_lane_exists[lane_n, :]

    def get_v_enc(self, v_n):
        return self.v_enc[v_n-1, :]
    
    def encode(self, batch):
        y = torch.empty(1, self.n_features, device='cuda')
        y[0, 0:3] = self.get_v_enc(batch[0])
        y[0, 3:5], le = self.get_left_lane_enc(batch[2])
        y[0, 5:], re = self.get_right_lane_enc(batch[3])
        
        return y, le*re
    
    def encode_batch(self, tensor):
        y = torch.empty(tensor.size(0), self.n_features, device='cuda')
        ex = torch.empty(tensor.size(0), 3, device='cuda')
        for batch in range(0, tensor.size(0)):
            y[batch, :], ex[batch, :] = self.encode(tensor[batch, :])
        return y, ex
    
    def forward(self, x):
        static_x = x[:, -1, 0:4].type(torch.int32)
        out, ex = self.encode_batch(static_x)
        return out, ex